In [16]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, vfx
import numpy as np


In [22]:
def refletir_imagem(frame):
    """
    Reflete horizontalmente o frame.
    """
    return frame[:, ::-1]

def aplicar_reflexao(video):
    """
    Aplica a reflexão horizontal alternada a cada 20 segundos.
    """
    clips = []
    for start in np.arange(0, video.duration, 20):
        end = min(start + 20, video.duration)
        subclip = video.subclip(start, end)
        if int(start // 20) % 2 == 1:  # Alternar reflexão
            subclip = subclip.fl_image(refletir_imagem)
        clips.append(subclip)
    return concatenate_videoclips(clips)

# Carregar o vídeo e aplicar reflexão
input_video_path = "video.mp4"
video = VideoFileClip(input_video_path)
video_refletido = aplicar_reflexao(video)



In [23]:
def reduzir_som_e_ficar_mudo(clip):
    """
    Reduz gradativamente o volume a cada 30 segundos e zera o som nos últimos 10 segundos.
    """
    audio = clip.audio
    if audio is None:
        print("Aviso: O vídeo não possui áudio para manipular.")
        return clip  # Retorna o vídeo original se não houver áudio

    duration = clip.duration
    fade_start = max(0, duration - 30)  # Início da redução gradativa
    mute_start = max(0, duration - 10)  # Início do silêncio total

    def ajustar_volume(gf, t):
        if np.isscalar(t):
            # Para tempos únicos
            if t >= mute_start:
                return 0 * gf(t)  # Ficar mudo nos últimos 10 segundos
            elif t >= fade_start:
                return (1 - (t - fade_start) / 20) * gf(t)  # Redução gradual
            else:
                return gf(t)
        else:
            # Para arrays de tempo
            fator = np.where(
                t >= mute_start, 
                0,  # Silêncio total
                np.where(t >= fade_start, 1 - (t - fade_start) / 20, 1)  # Gradual
            )
            return fator[:, None] * gf(t)

    # Aplicar a redução de som
    audio = audio.fl(ajustar_volume, keep_duration=True)
    return clip.set_audio(audio)


In [24]:
def cortar_e_inserir(video):
    """
    Corta o segmento do segundo 60 ao 80 e o insere ao final.
    """
    if video.duration > 80:
        corte = video.subclip(60, 80)
        video_final = concatenate_videoclips([video, corte])
    else:
        video_final = video
    return video_final

# Aplicar a redução de som
video_som_reduzido = reduzir_som_e_ficar_mudo(video_refletido)

# Continuar com o processamento final
video_final = cortar_e_inserir(video_som_reduzido)

# Salvar o vídeo final
output_video_path = "video_final.mp4"
video_final.write_videofile("video_final.mp4", codec="libx264", audio_codec="aac")


Moviepy - Building video video_final.mp4.
MoviePy - Writing audio in video_finalTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_final.mp4



Moviepy - Done !
Moviepy - video ready video_final.mp4
